In [1]:
import gym
import numpy as np
import tensorflow as tf


In [2]:
#CNN
def cnn(inputs_feed):
  """Model function for CNN."""
  mode = 'TRAIN'
  # Input Layer

  input_layer = tf.reshape(inputs_feed, [-1, 80,80, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=8,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=conv1,
      filters=16,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Dense Layer
  conv2_flat = tf.reshape(conv2, [-1, 80 * 80 * 16])
  dense = tf.layers.dense(inputs=conv2_flat, units=1024, activation=tf.nn.relu)
    
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  sigmoid = tf.layers.dense(inputs=dropout, units=1)
  up_probability = tf.nn.sigmoid(sigmoid) 

  return up_probability

#image pre processing

def downsample(image):
    # Take only alternate pixels - basically halves the resolution of the image (which is fine for us)
    return image[::2, ::2, :]#read a='hello' a[::2] = 'hlo'

def remove_color(image):
    """Convert all color (RGB is the third dimension in the image)"""
    return image[:, :, 0]

def remove_background(image):
    image[image == 144] = 0 #remove the number 144,list is not run , and np.array is ok a = np.array[12,23,144] result is a = [12,23,0]
    image[image == 109] = 0
    return image

def preprocess_observations(input_observation, prev_processed_observation, input_dimensions):
    """ convert the 210x160x3 uint8 frame into a 6400 float vector """
    processed_observation = input_observation[35:195] # crop
    processed_observation = downsample(processed_observation)
    processed_observation = remove_color(processed_observation)
    processed_observation = remove_background(processed_observation)
    processed_observation[processed_observation != 0] = 1 # everything else (paddles, ball) just set to 1
    # Convert from 80 x 80 matrix to 1600 x 1 matrix
    processed_observation = processed_observation.astype(np.float).ravel()

    # subtract the previous frame from the current one so we are only processing on changes in the game
    if prev_processed_observation is not None:
        input_observation = processed_observation - prev_processed_observation
    else:
        input_observation = np.zeros(input_dimensions)
    # store the previous frame so we can subtract from it next time
    prev_processed_observations = processed_observation
    #print input_observation
    return input_observation, prev_processed_observations

def choose_action(probability):
    random_value = np.random.uniform()
    if random_value < probability:
        # signifies up in openai gym
        return 2
    else:
         # signifies down in openai gym
        return 3
    # 0 is up and 1 is down
    # < 0.5 is up is 0
    

def weight_varible(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def prepro(I):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(np.float).ravel()

In [ ]:
# main

# hyperparameters
learning_rate = 0.001
episode_number = 0
number  = 0
batch_size = 2
observation =0
reward = 0
done = 0
info = 0 
reward_sum = 0
sum_of_probability = 0
D = 80 * 80 # input dimensionality: 80x80 grid
#1 get image and reward

#print a

# 2 process image
prev_processed_observations = None
input_dimensions = 80 * 80
#processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)
#print "1="
#print processed_observations
#print processed_observations.shape
#processed_observations = tf.cast(processed_observations, tf.float32)

# define a graph
graph1 = tf.Graph()
with graph1.as_default():
    env = gym.make("Pong-v0")
    observation = env.reset()
    prev_x = None # used in computing the difference frame
    print observation.shape
    processed_observation=observation[35:195]
    print processed_observation.shape

    a=remove_background(processed_observation)
    print a.shape
    ###################################################################
    #1 input
    x_ = tf.placeholder(shape=[None,6400],dtype=tf.float32)
    y_ = tf.placeholder(tf.float32, (None,1), name="fake_label")
    #reward_ =  tf.placeholder(shape=[1],dtype=tf.float32)
    reward_ = tf.placeholder(tf.float32, shape=(), name="reward_")
    #2 CNN(X)
    up_probability = cnn(x_)
    
    ##########################################
    '''
    """Model function for CNN."""
    mode = 'TRAIN'
    # Input Layer

    input_layer = tf.reshape(x_, [-1, 80,80, 1])
    # paras
    W_conv1 = weight_varible([5, 5, 1, 8])
    b_conv1 = bias_variable([8])
    
    h_conv1 = tf.nn.relu(conv2d(input_layer, W_conv1) + b_conv1)
    
    # conv layer-2
    W_conv2 = weight_varible([5, 5, 8, 16])
    b_conv2 = bias_variable([16])

    h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)

    # full connection
    W_fc1 = weight_varible([80 * 80 * 16, 1024])
    b_fc1 = bias_variable([1024])

    h_conv2_flat = tf.reshape(h_conv2, [-1, 80 * 80 * 16])
    h_fc1 = tf.nn.relu(tf.matmul(h_conv2_flat, W_fc1) + b_fc1)

    # dropout
    h_fc1_drop = tf.nn.dropout(h_fc1, 0.6)

    # output layer: softmax
    W_fc2 = weight_varible([1024, 1])
    b_fc2 = bias_variable([1])

    up_probability = tf.nn.sigmoid(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    '''
    ##########################################

    #3 cross_entropy
    #cost =  -1 * tf.exp(  tf.reduce_sum(  -1*( ( ( (1-y_)*tf.log(up_probability) + (y_) *tf.log(1-up_probability) ) ) + tf.log( reward_ )  )  )  ) 
    #cost =  tf.reduce_mean(  tf.reduce_sum( (1-y_)*tf.log(up_probability) + (y_) *tf.log(1-up_probability) ) )
    #cost = tf.reduce_mean(-tf.reduce_sum( (1-y_)*tf.log(up_probability + tf.log( reward_ ) )   , reduction_indices=1))
    #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
    cost = - ( tf.reduce_sum( (1-y_)*tf.log(up_probability) + (y_)* tf.log(1 - up_probability) + tf.log( reward_ ) ) )
    #4 Optimazor
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

    #5 xs_batch, reward_batch
    #
    ###############################################################################

    init = tf.global_variables_initializer()


    '''
    processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)
    #processed_observations = tf.cast(processed_observations, tf.float32)
    print "1="
    print processed_observations
    print processed_observations.shape
    print processed_observations.dtype
    processed_observations = [float(v) for v in processed_observations]
    '''
    
    ########################################################################################3
    # compute policy loss and regularization loss
    #copy from others
    #cross_entropy_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=up_probability, labels=taken_actions)
    #pg_loss  = tf.reduce_mean(self.cross_entropy_loss)
    #loss_with_reward = pg_loss * discounted_rewards

    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_with_reward)
    # compute gradients
    #gradients = self.optimizer.compute_gradients(self.loss)

    # compute policy gradients
    #for i, (grad, var) in enumerate(self.gradients):
    #if grad is not None:
    #    gradients[i] = (grad * self.discounted_rewards, var)
    ###########################################################################################

    #up_probability_from_cnn = sess.run(up_probability,feed_dict={inputs_feed: processed_observations})
    #Make the Magic happen
    #each_value_of_object_function =  up_probability_from_cnn # * reward
    '''
    ##################################################################################
    #Magic PG value * probability
    total_value_of_object_function = []
    total_value_of_object_function.append(each_value_of_object_function)
    # Minimize error using cross entropy
    #[tf.reduce_sum(tf.square(x)) for x in policy_network_variables]
    cost = tf.reduce_mean( -1 * reward_sum * np.sum(each_value_of_object_function, reduction_indices=1) for each_value_of_object_function in total_value_of_object_function)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    ###################################################################################
    '''
    #input to the CNN and get the output up_probability
    episode_processed_observations = []
    episode_rewards =  []
    episode_fake_label_action = []
    saver = tf.train.Saver()
    
    with tf.Session(graph=graph1) as sess:     

        init = tf.global_variables_initializer()
        sess.run(init)

        while True:
            env.render()


            # feed the CNN to get an action

            processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)

            processed_observations_np = np.reshape(processed_observations, [-1,6400])
            #print "1=" + str(processed_observations_np)
            #print "2=" + str(observation[0][10])
            #processed_observations = [0,float(v) for v in processed_observations]
            #print(processed_observations.shape)    
            
            # preprocess the observation, set input to network to be difference image
            #cur_x = prepro(observation)
            #print "2=" + str(observation)
            #print "prepro = " + str(cur_x)
            #for V in cur_x:
            #    if V > 0 :
            #        print V
            #x_input = cur_x - prev_x if prev_x is not None else np.zeros(D)
            #prev_x = cur_x
            
            #x_input = np.reshape(x_input, [-1,6400])
            #print "image = " + str(x_input[0][500])
            #counter = 0
            #for V in processed_observations_np[0]:
            #    counter += 1
            #    if V > 0 :
            #        print V
            #        print"counter = " + str(counter)
            #counter = 0
            
            up_probability_from_cnn = sess.run(up_probability,feed_dict={x_: processed_observations_np})
            #up_probability_from_cnn = [[0]]
            episode_processed_observations.append(processed_observations_np[0])
            
            
            
            print(up_probability_from_cnn)    
            #print("!!!!!!!!!!!!!!!!!!!!!!!")
            #print(np.shape(episode_processed_observations)) 
            #print(np.array(episode_processed_observations).shape)       


            action = choose_action(up_probability_from_cnn)
            
            #print("Action is ===========") + str(action)
            
            episode_fake_label_action.append( [action - 2] )
            
            #print(np.shape(episode_fake_label_action)) 
            
            # carry out the chosen action
            observation, reward, done, info = env.step(action)


            ############################
            #debug
            #print "reward = " + str(reward)
            
            reward_sum += reward
            #print "reward sum = " + str(reward_sum)


            number +=1
            #print "number = " + str(number)
            #print "episode_number = " + str(episode_number)
            #print "+++++++++++"

            #
            ####################################
            xs_batch = np.reshape( episode_processed_observations , [-1,6400])
            #print(xs_batch.shape)


            ##################################################################################
            #Magic PG value * probability
            total_value_of_object_function = []
            total_value_of_object_function.append(up_probability_from_cnn)

            if done: # an episode finished
                print "episode_number = " + str(episode_number)
                episode_number += 1

                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
               # print(episode_processed_observations)
                #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
                ###################################################################################
                #for i in range(100):
                #   optimizer.run()
                #  print "cost_goal: ",cost.eval()

                xs_batch = episode_processed_observations
                
                #print("length of xs batch =")
                #print(np.shape(xs_batch))
                
                ys_batch = episode_fake_label_action
                
                #print("length of ys_batch =")
                #print(np.shape(ys_batch))
                
                #xs_batch = xs_batch[:400]
                #reward_batch = reward_batch[:400]
                #print(np.shape(xs_batch))
                #print np.array(reward_batch).shape
                
                num= len(episode_processed_observations) / 300
                
                if num%300 == 0 :
                    continue
                #print("number of batch = ") + str(num)
                reward_sum += 23
                
                for n in range(num+1):
                    xs_batch_temp = xs_batch[n*300:(n+1)*300]
                    ys_batch_temp = ys_batch[n*300:(n+1)*300]
                    if n == num and len(episode_processed_observations) != n*300 :
                        xs_batch_temp = xs_batch[n*300:]
                        ys_batch_temp = ys_batch[n*300:]
                    
                    #print("the traning data shape is =")
                    print(np.shape(xs_batch_temp))
                    print(np.shape(ys_batch_temp))
                

                    
                    _,cost_result = sess.run( [train_step, cost], feed_dict={x_: xs_batch_temp, y_ : ys_batch_temp, reward_: reward_sum} )
                    print("the run time for once COST = ") + str(cost_result)
                    print "reward sum = " + str(reward_sum)
                counter = 1
                print("SAVED")
                #save_path = saver.save(sess, "/home/hadoop/DeepReinforcementLearning/model.ckpt")
                if episode_number % batch_size == 0:
                    if counter == 1:
                        saver.restore(sess, "/home/hadoop/DeepReinforcementLearning/model.ckpt")
                        counter +=1
                    save_path = saver.save(sess, "/home/hadoop/DeepReinforcementLearning/model.ckpt")
                    saver.restore(sess, "/home/hadoop/DeepReinforcementLearning/model.ckpt")
                    
                    
                reward_sum = 0
                xs_batch=[]
                ys_batch=[]
                reward_batch = []
                episode_processed_observations =[]
                episode_fake_label_action = []
                prev_processed_observations = None
                observation = env.reset() # reset env


    #_, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,y: batch_ys})

[2018-02-01 17:32:03,689] Making new env: Pong-v0


(210, 160, 3)
(160, 160, 3)
(160, 160, 3)
[[0.5]]
[[0.49647963]]
[[0.499504]]
[[0.5]]
[[0.4937829]]
[[0.49751592]]
[[0.4974122]]
[[0.49786872]]
[[0.499504]]
[[0.49365538]]
[[0.5]]
[[0.49792618]]
[[0.49917215]]
[[0.49967656]]
[[0.49780905]]
[[0.49945033]]
[[0.49879432]]
[[0.5015874]]
[[0.49589744]]
[[0.5001872]]
[[0.5020372]]
[[0.499688]]
[[0.49821183]]
[[0.4900232]]
[[0.49783173]]
[[0.4967741]]
[[0.5004327]]
[[0.4972007]]
[[0.49928778]]
[[0.49785537]]
[[0.4919437]]
[[0.5000607]]
[[0.4963522]]
[[0.49392837]]
[[0.49576402]]
[[0.4981612]]
[[0.49771252]]
[[0.49585623]]
[[0.49742022]]
[[0.49820653]]
[[0.5015746]]
[[0.4993754]]
[[0.49646005]]
[[0.49224973]]
[[0.49369287]]
[[0.49559662]]
[[0.49031362]]
[[0.49690023]]
[[0.4941236]]
[[0.5005466]]
[[0.49339005]]
[[0.50332594]]
[[0.49609774]]
[[0.50159997]]
[[0.49949363]]
[[0.49602935]]
[[0.49624142]]
[[0.48761508]]
[[0.49369135]]
[[0.4976778]]
[[0.49310386]]
[[0.496501]]
[[0.4977334]]
[[0.49776047]]
[[0.4974854]]
[[0.50345755]]
[[0.49915218]]
[[

[[0.4961947]]
[[0.49799722]]
[[0.49006093]]
[[0.49506703]]
[[0.49597847]]
[[0.4963684]]
[[0.496249]]
[[0.50360775]]
[[0.49898496]]
[[0.49805447]]
[[0.49680465]]
[[0.4970506]]
[[0.49663287]]
[[0.49991858]]
[[0.4969338]]
[[0.49390128]]
[[0.5013477]]
[[0.49856895]]
[[0.49614576]]
[[0.4923018]]
[[0.4954626]]
[[0.494874]]
[[0.4984318]]
[[0.4937209]]
[[0.4995599]]
[[0.49642032]]
[[0.49884337]]
[[0.5004772]]
[[0.49698743]]
[[0.49456385]]
[[0.49983817]]
[[0.49732733]]
[[0.5015668]]
[[0.49765304]]
[[0.5000229]]
[[0.4961273]]
[[0.499168]]
[[0.49582005]]
[[0.4960333]]
[[0.49458098]]
[[0.49266136]]
[[0.4936796]]
[[0.50125754]]
[[0.49205813]]
[[0.50497615]]
[[0.4972018]]
[[0.5014904]]
[[0.5042416]]
[[0.49939823]]
[[0.5068438]]
[[0.49833685]]
[[0.494354]]
[[0.5022858]]
[[0.49626914]]
[[0.50117]]
[[0.49815297]]
[[0.49578172]]
[[0.49559128]]
[[0.5014673]]
[[0.4966504]]
[[0.49379662]]
[[0.49804857]]
[[0.4972314]]
[[0.49842048]]
[[0.49424693]]
[[0.4979049]]
[[0.49933848]]
[[0.4979102]]
[[0.5013469]]
[[0

[[0.49584568]]
[[0.5018912]]
[[0.4959804]]
[[0.5003957]]
[[0.49700868]]
[[0.499033]]
[[0.49379703]]
[[0.49847272]]
[[0.50025696]]
[[0.49671087]]
[[0.49779776]]
[[0.49617445]]
[[0.49500453]]
[[0.49891067]]
[[0.50036365]]
[[0.50331914]]
[[0.50150454]]
[[0.49965525]]
[[0.49485934]]
[[0.49714735]]
[[0.4907404]]
[[0.49896342]]
[[0.49210727]]
[[0.49727827]]
[[0.49468726]]
[[0.501041]]
[[0.49545264]]
[[0.49858436]]
[[0.4956481]]
[[0.4958801]]
[[0.49872693]]
[[0.49482232]]
[[0.49570712]]
[[0.5000094]]
[[0.49694678]]
[[0.49610126]]
[[0.4966603]]
[[0.4988556]]
[[0.49678063]]
[[0.49463803]]
[[0.4910009]]
[[0.49908328]]
[[0.49730107]]
[[0.5001826]]
[[0.49914923]]
[[0.49802527]]
[[0.49859443]]
[[0.4966884]]
[[0.49909934]]
[[0.4985852]]
[[0.49374303]]
[[0.4923382]]
[[0.49843043]]
[[0.50014216]]
[[0.4955075]]
[[0.49920702]]
[[0.49424565]]
[[0.4990972]]
[[0.50213885]]
[[0.5005242]]
[[0.5012497]]
[[0.5002995]]
[[0.50115573]]
[[0.49563903]]
[[0.49737963]]
[[0.49183086]]
[[0.49687234]]
[[0.49957812]]
[[0

[[0.50214034]]
[[0.50722903]]
[[0.5067227]]
[[0.50550187]]
[[0.50668323]]
[[0.5019128]]
[[0.5067364]]
[[0.5064663]]
[[0.50935763]]
[[0.5122936]]
[[0.5105255]]
[[0.5116064]]
[[0.5060278]]
[[0.5146805]]
[[0.51038694]]
[[0.5098165]]
[[0.51016206]]
[[0.51622146]]
[[0.50123185]]
[[0.5116825]]
[[0.5100162]]
[[0.5128332]]
[[0.51238495]]
[[0.5060984]]
[[0.50851136]]
[[0.5068823]]
[[0.5031391]]
[[0.5078776]]
[[0.5017884]]
[[0.5055641]]
[[0.51029265]]
[[0.5042342]]
[[0.50562805]]
[[0.5028135]]
[[0.51281106]]
[[0.5093011]]
[[0.51183385]]
[[0.51493216]]
[[0.5109666]]
[[0.5138078]]
[[0.5008597]]
[[0.5127761]]
[[0.5110306]]
[[0.5158984]]
[[0.50647396]]
[[0.5135603]]
[[0.5055498]]
[[0.5125003]]
[[0.5119917]]
[[0.51586175]]
[[0.5045093]]
[[0.5116545]]
[[0.52059585]]
[[0.5176824]]
[[0.5142634]]
[[0.5101164]]
[[0.5057186]]
[[0.51292205]]
[[0.51133394]]
[[0.51475036]]
[[0.5074721]]
[[0.5118509]]
[[0.50967675]]
[[0.5129495]]
[[0.5106389]]
[[0.5171147]]
[[0.51219493]]
[[0.5102859]]
[[0.5093918]]
[[0.512418

...

[[0.521053]]
[[0.5155608]]
[[0.5149797]]
[[0.5218751]]
[[0.5142201]]
[[0.5169575]]
[[0.5173063]]
[[0.51624984]]
[[0.51740617]]
[[0.5186202]]
[[0.50947535]]
[[0.5192181]]
[[0.52372754]]
[[0.5183014]]
[[0.5228008]]
[[0.5193836]]
[[0.5164262]]
[[0.5166917]]
[[0.5210243]]
[[0.5182601]]
[[0.521422]]
[[0.5226875]]
[[0.5173628]]
[[0.5187288]]
[[0.5193295]]
[[0.51633716]]
[[0.51330525]]
[[0.5097366]]
[[0.51319736]]
[[0.5134561]]
[[0.51809126]]
[[0.5202566]]
[[0.5172342]]
[[0.5171191]]
[[0.5207212]]
[[0.52107406]]
[[0.5172946]]
[[0.5161231]]
[[0.5175622]]
[[0.5180788]]
[[0.51287746]]
[[0.51295173]]
[[0.51074773]]
[[0.51175]]
[[0.5179865]]
[[0.514488]]
[[0.5139322]]
[[0.51633817]]
[[0.5187207]]
[[0.512644]]
[[0.51586384]]
[[0.5176711]]
[[0.5172929]]
[[0.51662004]]
[[0.5187678]]
[[0.5154444]]
[[0.51504207]]
[[0.51895654]]
[[0.5151705]]
[[0.5146277]]
[[0.5128372]]
[[0.51385516]]
[[0.50243217]]
[[0.51400864]]
[[0.5149546]]
[[0.512237]]
[[0.507457]]
[[0.5118094]]
[[0.5119709]]
[[0.5092253]]
[[0.5190

[[0.5133566]]
[[0.50451005]]
[[0.51257324]]
[[0.51196826]]
[[0.5101232]]
[[0.51178217]]
[[0.5100982]]
[[0.5144713]]
[[0.5127972]]
[[0.511085]]
[[0.5140325]]
[[0.51408553]]
[[0.5160704]]
[[0.51458335]]
[[0.511011]]
[[0.5123425]]
[[0.5157402]]
[[0.5179433]]
[[0.5186381]]
[[0.51707786]]
[[0.50884765]]
[[0.51820785]]
[[0.51378137]]
[[0.5106629]]
[[0.51190656]]
[[0.509004]]
[[0.51186264]]
[[0.51215374]]
[[0.5126974]]
[[0.51140136]]
[[0.5139952]]
[[0.50780743]]
[[0.5118822]]
[[0.50958854]]
[[0.5157772]]
[[0.51564527]]
[[0.51153404]]
[[0.5156033]]
[[0.5138188]]
[[0.51009804]]
[[0.5158654]]
[[0.5133753]]
[[0.51411796]]
[[0.5133911]]
[[0.512471]]
[[0.51173025]]
[[0.506627]]
[[0.5086251]]
[[0.51363206]]
[[0.5163537]]
[[0.5162092]]
[[0.51639163]]
[[0.511365]]
[[0.5136802]]
[[0.5103665]]
[[0.51720613]]
[[0.5103178]]
[[0.5191216]]
[[0.51660556]]
[[0.5127993]]
[[0.5085263]]
[[0.5106584]]
[[0.5147819]]
[[0.5137427]]
[[0.5146948]]
[[0.5161266]]
[[0.5188436]]
[[0.51764226]]
[[0.5091417]]
[[0.5155678]]


[[0.51545036]]
[[0.5152055]]
[[0.51965326]]
[[0.5109242]]
[[0.51460737]]
[[0.5104549]]
[[0.51178396]]
[[0.5067393]]
[[0.5112625]]
[[0.5102444]]
[[0.50999486]]
[[0.5135861]]
[[0.51140136]]
[[0.5145162]]
[[0.5079464]]
[[0.5113151]]
[[0.5156106]]
[[0.50905126]]
[[0.5148862]]
[[0.5139312]]
[[0.5148923]]
[[0.5132799]]
[[0.5116017]]
[[0.5148391]]
[[0.51592094]]
[[0.50973004]]
[[0.5105617]]
[[0.50618047]]
[[0.51180744]]
[[0.5079919]]
[[0.5090634]]
[[0.5052975]]
[[0.50866383]]
[[0.5116079]]
[[0.5130613]]
[[0.5088968]]
[[0.5136049]]
[[0.5114856]]
[[0.5099671]]
[[0.5131771]]
[[0.51018655]]
[[0.5145664]]
[[0.51382107]]
[[0.51390386]]
[[0.51005304]]
[[0.51595926]]
[[0.51733124]]
[[0.51281995]]
[[0.51506066]]
[[0.5179727]]
[[0.5127537]]
[[0.51253814]]
[[0.5152084]]
[[0.50790745]]
[[0.5075049]]
[[0.5136403]]
[[0.5149667]]
[[0.51106757]]
[[0.5103339]]
[[0.51372385]]
[[0.5113627]]
[[0.5135601]]
[[0.5138285]]
[[0.510533]]
[[0.51646066]]
[[0.5127403]]
[[0.512477]]
[[0.5147443]]
[[0.5131205]]
[[0.5151371

[[0.5107131]]
[[0.51765937]]
[[0.51441157]]
[[0.51934576]]
[[0.52010256]]
[[0.5235345]]
[[0.5187569]]
[[0.5101111]]
[[0.5228406]]
[[0.5266011]]
[[0.51375705]]
[[0.5114406]]
[[0.50794935]]
[[0.5122548]]
[[0.5267577]]
[[0.5185125]]
[[0.5146596]]
[[0.5098645]]
[[0.51331526]]
[[0.51925534]]
[[0.52451164]]
[[0.51562744]]
[[0.5177955]]
[[0.5156077]]
[[0.515054]]
[[0.516433]]
[[0.5139636]]
[[0.52300525]]
[[0.523261]]
[[0.52170193]]
[[0.5257698]]
[[0.52090085]]
[[0.5202361]]
[[0.5161208]]
[[0.5158391]]
[[0.51424664]]
[[0.51587677]]
[[0.5201229]]
[[0.5205321]]
[[0.51171887]]
[[0.51892686]]
[[0.50864786]]
[[0.5132261]]
[[0.50636804]]
[[0.5146297]]
[[0.51480526]]
[[0.51003516]]
[[0.5126074]]
[[0.50948256]]
[[0.5101503]]
[[0.51726085]]
[[0.5101903]]
[[0.51460886]]
[[0.5162204]]
[[0.5152508]]
[[0.51765996]]
[[0.51349574]]
[[0.51740927]]
[[0.51263446]]
[[0.5130503]]
[[0.51586217]]
[[0.51513314]]
[[0.5120752]]
[[0.510593]]
[[0.5052128]]
[[0.50675154]]
[[0.508733]]
[[0.5063752]]
[[0.5079098]]
[[0.5117

[[0.5085333]]
[[0.51225156]]
[[0.49954414]]
[[0.5035455]]
[[0.5058658]]
[[0.510475]]
[[0.510475]]
[[0.510475]]
[[0.50807625]]
[[0.50863206]]
[[0.505915]]
[[0.5074497]]
[[0.510475]]
[[0.5141568]]
[[0.51167244]]
[[0.50216776]]
[[0.5076017]]
[[0.5141854]]
[[0.5098654]]
[[0.507632]]
[[0.507079]]
[[0.50968343]]
[[0.51210755]]
[[0.5200991]]
[[0.5082165]]
[[0.50910836]]
[[0.5186041]]
[[0.509751]]
[[0.5125356]]
[[0.51695603]]
[[0.5163504]]
[[0.5155936]]
[[0.51770175]]
[[0.5116732]]
[[0.50474423]]
[[0.50650716]]
[[0.5076193]]
[[0.5098947]]
[[0.5081002]]
[[0.52853376]]
[[0.52350986]]
[[0.51000047]]
[[0.50963706]]
[[0.51434916]]
[[0.51006025]]
[[0.51022923]]
[[0.51493555]]
[[0.5146251]]
[[0.5198497]]
[[0.510475]]
[[0.5152496]]
[[0.5202067]]
[[0.52288187]]
[[0.5166904]]
[[0.51149416]]
[[0.51007944]]
[[0.510475]]
[[0.5120923]]
[[0.51085985]]
[[0.521575]]
[[0.5150093]]
[[0.51191884]]
[[0.5101005]]
[[0.5184367]]
[[0.5135104]]
[[0.50964075]]
[[0.5110238]]
[[0.51383334]]
[[0.50636154]]
[[0.5048328]]
[[

In [14]:
1, -277.28204

4164.5967

In [9]:
# main

# hyperparameters
learning_rate = 0.01
episode_number = 0
number  = 0
batch_size = 3
observation =0
reward = 0
done = 0
info = 0 
reward_sum = 0
sum_of_probability = 0
#1 get image and reward
env = gym.make("Pong-v0")
observation = env.reset()
print observation.shape
processed_observation=observation[35:195]
print processed_observation.shape

a=remove_background(processed_observation)
print a.shape
#print a

# 2 process image
prev_processed_observations = None
input_dimensions = 80 * 80
#processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)
#print "1="
#print processed_observations
#print processed_observations.shape
#processed_observations = tf.cast(processed_observations, tf.float32)

###################################################################
inputs_feed = tf.placeholder(shape=[6400],dtype=tf.float32)

init = tf.global_variables_initializer()

processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)
#processed_observations = tf.cast(processed_observations, tf.float32)
print "1="
print processed_observations
print processed_observations.shape
print processed_observations.dtype
processed_observations = [float(v) for v in processed_observations]
up_probability = cnn(inputs_feed)


#up_probability_from_cnn = sess.run(up_probability,feed_dict={inputs_feed: processed_observations})
#Make the Magic happen
#each_value_of_object_function =  up_probability_from_cnn # * reward
'''
##################################################################################
#Magic PG value * probability
total_value_of_object_function = []
total_value_of_object_function.append(each_value_of_object_function)
# Minimize error using cross entropy
#[tf.reduce_sum(tf.square(x)) for x in policy_network_variables]
cost = tf.reduce_mean( -1 * reward_sum * np.sum(each_value_of_object_function, reduction_indices=1) for each_value_of_object_function in total_value_of_object_function)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
###################################################################################
'''
#input to the CNN and get the output up_probability

with tf.Session() as sess:     
    
    init = tf.global_variables_initializer()
    sess.run(init)
    
    while True:
        env.render()

        print "2="
        up_probability_from_cnn = sess.run(up_probability,feed_dict={inputs_feed: processed_observations})
        print(up_probability_from_cnn)    
        


        action = choose_action(up_probability_from_cnn)
        # carry out the chosen action
        observation, reward, done, info = env.step(action)
        episode_rewards =  []
        
        ############################
        #debug
        print "reward = " + str(reward)
        reward_sum += reward
        print "reward sum = " + str(reward_sum)
        episode_rewards.append(reward)
        number +=1
        print "number = " + str(number)
        print "episode_number = " + str(episode_number)
        print "+++++++++++"
        #
        ####################################
        
        ##################################################################################
        #Magic PG value * probability
        total_value_of_object_function = []
        total_value_of_object_function.append(up_probability_from_cnn)

        sum_of_probability += up_probability_from_cnn

        if done: # an episode finished
            episode_number += 1


            optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
            ###################################################################################
            for i in range(100):
                optimizer.run()
                print "cost_goal: ",cost.eval()
            
            
            
            if episode_number % batch_size == 0:
                break
                
            observation = env.reset() # reset env
            reward_sum = 0
            prev_processed_observations = None

#_, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,y: batch_ys})

[2018-01-30 14:37:42,505] Making new env: Pong-v0


(210, 160, 3)
(160, 160, 3)
(160, 160, 3)
1=
[0. 0. 0. ... 0. 0. 0.]
(6400,)
float64
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 1
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 2
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 3
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 4
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 5
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 6
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 7
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 8
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 9
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 10
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 11
episode_number = 0
+++++++++++


[[0.5]]
reward = 0.0
reward sum = 0.0
number = 100
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 101
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 102
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 103
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 104
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 105
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 106
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 107
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 108
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 109
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 110
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 111
episode_number

[[0.5]]
reward = 0.0
reward sum = -1.0
number = 202
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -1.0
number = 203
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -1.0
number = 204
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -1.0
number = 205
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -1.0
number = 206
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -1.0
number = 207
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -1.0
number = 208
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -1.0
number = 209
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -1.0
number = 210
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -1.0
number = 211
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -1.0
number = 212
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -1.0
number = 213
ep

[[0.5]]
reward = 0.0
reward sum = -2.0
number = 302
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -2.0
number = 303
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -2.0
number = 304
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -2.0
number = 305
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -2.0
number = 306
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -2.0
number = 307
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -2.0
number = 308
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -2.0
number = 309
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -2.0
number = 310
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -2.0
number = 311
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -2.0
number = 312
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -2.0
number = 313
ep

...

[[0.5]]
reward = 0.0
reward sum = -16.0
number = 1476
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -16.0
number = 1477
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -16.0
number = 1478
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -16.0
number = 1479
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -16.0
number = 1480
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -16.0
number = 1481
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -16.0
number = 1482
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -16.0
number = 1483
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -16.0
number = 1484
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -16.0
number = 1485
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -16.0
number = 1486
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum 

[[0.5]]
reward = 0.0
reward sum = -17.0
number = 1572
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -17.0
number = 1573
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -17.0
number = 1574
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -17.0
number = 1575
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -17.0
number = 1576
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -17.0
number = 1577
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -17.0
number = 1578
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -17.0
number = 1579
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -17.0
number = 1580
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -17.0
number = 1581
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -17.0
number = 1582
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum 

[[0.5]]
reward = 0.0
reward sum = -18.0
number = 1667
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -18.0
number = 1668
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -18.0
number = 1669
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -18.0
number = 1670
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -18.0
number = 1671
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -18.0
number = 1672
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -18.0
number = 1673
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -18.0
number = 1674
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -18.0
number = 1675
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -18.0
number = 1676
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -18.0
number = 1677
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum 

[[0.5]]
reward = 0.0
reward sum = -19.0
number = 1763
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -19.0
number = 1764
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -19.0
number = 1765
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -19.0
number = 1766
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -19.0
number = 1767
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -19.0
number = 1768
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -19.0
number = 1769
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -19.0
number = 1770
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -19.0
number = 1771
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -19.0
number = 1772
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = -19.0
number = 1773
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum 

ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'conv2d/kernel:0' shape=(5, 5, 1, 8) dtype=float32_ref>", "<tf.Variable 'conv2d/bias:0' shape=(8,) dtype=float32_ref>", "<tf.Variable 'conv2d_1/kernel:0' shape=(5, 5, 8, 16) dtype=float32_ref>", "<tf.Variable 'conv2d_1/bias:0' shape=(16,) dtype=float32_ref>", "<tf.Variable 'dense/kernel:0' shape=(102400, 1024) dtype=float32_ref>", "<tf.Variable 'dense/bias:0' shape=(1024,) dtype=float32_ref>", "<tf.Variable 'dense_1/kernel:0' shape=(1024, 1) dtype=float32_ref>", "<tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32_ref>", "<tf.Variable 'conv2d_2/kernel:0' shape=(5, 5, 1, 8) dtype=float32_ref>", "<tf.Variable 'conv2d_2/bias:0' shape=(8,) dtype=float32_ref>", "<tf.Variable 'conv2d_3/kernel:0' shape=(5, 5, 8, 16) dtype=float32_ref>", "<tf.Variable 'conv2d_3/bias:0' shape=(16,) dtype=float32_ref>", "<tf.Variable 'dense_2/kernel:0' shape=(102400, 102

In [1]:
# main

# hyperparameters
learning_rate = 0.01
episode_number = 0
number  = 0
batch_size = 20
observation =0
reward = 0
done = 0
info = 0 
reward_sum = 0

#1 get image and reward
env = gym.make("Pong-v0")
observation = env.reset()
print observation.shape
processed_observation=observation[35:195]
print processed_observation.shape

a=remove_background(processed_observation)
print a.shape
#print a

# 2 process image
prev_processed_observations = None
input_dimensions = 80 * 80
#processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)
#print "1="
#print processed_observations
#print processed_observations.shape
#processed_observations = tf.cast(processed_observations, tf.float32)

###################################################################
inputs_feed = tf.placeholder(shape=[6400],dtype=tf.float32)

init = tf.global_variables_initializer()

processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)
#processed_observations = tf.cast(processed_observations, tf.float32)
print "1="
print processed_observations
print processed_observations.shape
print processed_observations.dtype
processed_observations = [float(v) for v in processed_observations]

up_probability = cnn(inputs_feed)
#input to the CNN and get the output up_probability
with tf.Session() as sess:     


    print "3="
    print up_probability
    #print the product
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(up_probability,feed_dict={inputs_feed: processed_observations}))    
    #print(up_probability.eval()[0][0])
    
#Make the Magic happen
each_value_of_object_function =  up_probability  * reward
# Minimize error using cross entropy
cost = tf.reduce_mean(-1*tf.reduce_sum(each_value_of_object_function, reduction_indices=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

#_, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,y: batch_ys})
'''
init = tf.global_variables_initializer()

processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)
processed_observations = tf.cast(processed_observations, tf.float32)
up_probability = cnn()



with tf.Session() as sess:     

    sess.run(init)
    
    while True:
        env.render()

        
       #up_probability = cnn(processed_observations)
        print "3="
        print episode_number
        #rint up_probability
        #print the product
        print(sess.run(up_probability,feed_dict=processed_observations))    
        print(up_probability.eval()[0][0])
        
        action = choose_action(up_probability.eval()[0][0])
        # carry out the chosen action
        observation, reward, done, info = env.step(action)
        episode_rewards =  []

        reward_sum += reward
        episode_rewards.append(reward)

        number +=1
        print number
        print "==========="
        print episode_number
        print "+++++++++++"

        if done: # an episode finished
            episode_number += 1

            #if episode_number % batch_size == 0:

                #break
            observation = env.reset() # reset env
'''

NameError: name 'gym' is not defined

In [ ]:
    
    with tf.Session() as sess:     

       #up_probability = cnn(processed_observations)
        print "3="
        print episode_number
        print up_probability
        #print the product
        init = tf.global_variables_initializer()
        sess.run(init)
        print(sess.run(up_probability))    
        print(up_probability.eval()[0][0])

        action = choose_action(up_probability.eval()[0][0])
        # carry out the chosen action
        observation, reward, done, info = env.step(action)
        episode_rewards =  []

        reward_sum += reward
        episode_rewards.append(reward)

In [3]:
#CNN
def cnn(inputs_feed):
  """Model function for CNN."""
  mode = 'TRAIN'
  # Input Layer

  input_layer = tf.reshape(inputs_feed, [-1, 80,80, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=8,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=conv1,
      filters=16,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Dense Layer
  conv2_flat = tf.reshape(conv2, [-1, 80 * 80 * 16])
  dense = tf.layers.dense(inputs=conv2_flat, units=1024, activation=tf.nn.relu)
    
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  sigmoid = tf.layers.dense(inputs=dropout, units=1)
  up_probability = tf.nn.sigmoid(sigmoid)

  return up_probability

In [7]:
a=[]
a.append(1)
a.append(2)
a.append(3)
print np.vstack(a)
print np.vstack(a)
print np.vstack(a)
print a
print np.vstack(a) * np.vstack(a)

[[1]
 [2]
 [3]]
[[1]
 [2]
 [3]]
[[1]
 [2]
 [3]]
[1, 2, 3]
[[1]
 [4]
 [9]]


In [4]:
#image pre processing

def downsample(image):
    # Take only alternate pixels - basically halves the resolution of the image (which is fine for us)
    return image[::2, ::2, :]#read a='hello' a[::2] = 'hlo'

def remove_color(image):
    """Convert all color (RGB is the third dimension in the image)"""
    return image[:, :, 0]

def remove_background(image):
    image[image == 144] = 0 #remove the number 144,list is not run , and np.array is ok a = np.array[12,23,144] result is a = [12,23,0]
    image[image == 109] = 0
    return image

def preprocess_observations(input_observation, prev_processed_observation, input_dimensions):
    """ convert the 210x160x3 uint8 frame into a 6400 float vector """
    processed_observation = input_observation[35:195] # crop
    processed_observation = downsample(processed_observation)
    processed_observation = remove_color(processed_observation)
    processed_observation = remove_background(processed_observation)
    processed_observation[processed_observation != 0] = 1 # everything else (paddles, ball) just set to 1
    # Convert from 80 x 80 matrix to 1600 x 1 matrix
    processed_observation = processed_observation.astype(np.float).ravel()

    # subtract the previous frame from the current one so we are only processing on changes in the game
    if prev_processed_observation is not None:
        input_observation = processed_observation - prev_processed_observation
    else:
        input_observation = np.zeros(input_dimensions)
    # store the previous frame so we can subtract from it next time
    prev_processed_observations = processed_observation
    return input_observation, prev_processed_observations

def choose_action(probability):
    random_value = np.random.uniform()
    if random_value < probability:
        # signifies up in openai gym
        return 2
    else:
         # signifies down in openai gym
        return 3

In [ ]:
#CNN create

def create_CNN(input_observation):
      """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(input_observation, [-1, 80, 80, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  #pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=conv1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  #pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(conv2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    return 

In [ ]:
#main loop

In [9]:

def downsample(image):
    # Take only alternate pixels - basically halves the resolution of the image (which is fine for us)
    return image[::2, ::2, :]#read a='hello' a[::2] = 'hlo'

def remove_color(image):
    """Convert all color (RGB is the third dimension in the image)"""
    return image[:, :, 0]

def remove_background(image):
    image[image == 144] = 0 #remove the number 144,list is not run , and np.array is ok a = np.array[12,23,144] result is a = [12,23,0]
    image[image == 109] = 0
    return image

def preprocess_observations(input_observation, prev_processed_observation, input_dimensions):
    """ convert the 210x160x3 uint8 frame into a 6400 float vector """
    processed_observation = input_observation[35:195] # crop
    processed_observation = downsample(processed_observation)
    processed_observation = remove_color(processed_observation)
    processed_observation = remove_background(processed_observation)
    processed_observation[processed_observation != 0] = 1 # everything else (paddles, ball) just set to 1
    # Convert from 80 x 80 matrix to 1600 x 1 matrix
    processed_observation = processed_observation.astype(np.float).ravel()

    # subtract the previous frame from the current one so we are only processing on changes in the game
    if prev_processed_observation is not None:
        input_observation = processed_observation - prev_processed_observation
    else:
        input_observation = np.zeros(input_dimensions)
    # store the previous frame so we can subtract from it next time
    prev_processed_observations = processed_observation
    return input_observation, prev_processed_observations


def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def relu(vector):
    vector[vector < 0] = 0
    return vector

def apply_neural_nets(observation_matrix, weights):
    """ Based on the observation_matrix and weights, compute the new hidden layer values and the new output layer values"""
    hidden_layer_values = np.dot(weights['1'], observation_matrix)
    hidden_layer_values = relu(hidden_layer_values)
    output_layer_values = np.dot(hidden_layer_values, weights['2'])
    output_layer_values = sigmoid(output_layer_values)
    return hidden_layer_values, output_layer_values

def choose_action(probability):
    random_value = np.random.uniform()
    if random_value < probability:
        # signifies up in openai gym
        return 2
    else:
         # signifies down in openai gym
        return 3

def compute_gradient(gradient_log_p, hidden_layer_values, observation_values, weights):
    """ See here: http://neuralnetworksanddeeplearning.com/chap2.html"""
    delta_L = gradient_log_p
    dC_dw2 = np.dot(hidden_layer_values.T, delta_L).ravel()
    delta_l2 = np.outer(delta_L, weights['2'])
    delta_l2 = relu(delta_l2)
    dC_dw1 = np.dot(delta_l2.T, observation_values)
    return {
        '1': dC_dw1,
        '2': dC_dw2
    }

def update_weights(weights, expectation_g_squared, g_dict, decay_rate, learning_rate):
    """ See here: http://sebastianruder.com/optimizing-gradient-descent/index.html#rmsprop"""
    epsilon = 1e-5
    for layer_name in weights.keys():
        g = g_dict[layer_name]
        expectation_g_squared[layer_name] = decay_rate * expectation_g_squared[layer_name] + (1 - decay_rate) * g**2
        weights[layer_name] += (learning_rate * g)/(np.sqrt(expectation_g_squared[layer_name] + epsilon))
        g_dict[layer_name] = np.zeros_like(weights[layer_name]) # reset batch gradient buffer

def discount_rewards(rewards, gamma):
    """ Actions you took 20 steps before the end result are less important to the overall result than an action you took a step ago.
    This implements that logic by discounting the reward on previous actions based on how long ago they were taken"""
    discounted_rewards = np.zeros_like(rewards)
    running_add = 0
    for t in reversed(xrange(0, rewards.size)):
        if rewards[t] != 0:
            running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + rewards[t]
        discounted_rewards[t] = running_add
    return discounted_rewards

def discount_with_rewards(gradient_log_p, episode_rewards, gamma):
    """ discount the gradient with the normalized rewards """
    discounted_episode_rewards = discount_rewards(episode_rewards, gamma)
    # standardize the rewards to be unit normal (helps control the gradient estimator variance)
    discounted_episode_rewards -= np.mean(discounted_episode_rewards)
    discounted_episode_rewards /= np.std(discounted_episode_rewards)
    return gradient_log_p * discounted_episode_rewards

In [ ]:
## Architecture

# Take in inputs from the screen and preprocess them
# Pass them into an NN
# Update the weights of the NN using gradient descent
# weights['1'] - Matrix that holds weights of pixels passing into hidden layer. Dimensions: [200 x 80 x 80] -> [200 x 6400]
# weights['2'] - Matrix that holds weights of hidden layer passing into output. Dimensions: [1 x 200]

# Process is:

# processed_observations = image vector - [6400 x 1] array
# Compute hidden_layer_values = weights['1'] dot processed_observations ([200 x 6400] dot [6400 x 1]) -> [200 x 1] - this gives initial activation values.
# Next we need to transform those either via a sigmoid or an ReLU of some sort. Let's use ReLU
# ReLU(hidden_layer_values)
# Next we need to pass this one layer further
# output_layer_value = weights['2'] dot hidden_layer_values ([1 x 200] dot [200 x 1] -> [1 x 1])
# Now our output layer is the probability of going up or down. Let's make sure this output is between 0 and 1 by passing it through a sigmoid
# p = sigmoid(output_layer_value)

# Learning after round has finished:

# Figure out the result
# Compute the error
# Use the error to calculate the gradient
    # The below dimensions all assume we had exactly 10 frames in the round (not necessarily true!)
    # dC_dw2 = hidden_layer_values^T dot gradient_log_p ([1 x 2000] dot [2000 x 1] -> 1x1)
    # delta_1 = gradient_log_p outer_product weights['2'] = [2000 x 1] outer_product [1 x 200] ([2000 x 200])
    # dC_dw1 = delta_1^T dot input_observations ([200 x 2000]x dot [2000 x 64000] -> [200 x 64000])

# After some batch size of rounds has finished,
    # Use rmsprop to move weights['1'] and weights['2'] in the direction of the gradient
# Repeat!

import gym
import numpy as np

def downsample(image):
    # Take only alternate pixels - basically halves the resolution of the image (which is fine for us)
    return image[::2, ::2, :]#read a='hello' a[::2] = 'hlo'

def remove_color(image):
    """Convert all color (RGB is the third dimension in the image)"""
    return image[:, :, 0]

def remove_background(image):
    image[image == 144] = 0 #remove the number 144,list is not run , and np.array is ok a = np.array[12,23,144] result is a = [12,23,0]
    image[image == 109] = 0
    return image

def preprocess_observations(input_observation, prev_processed_observation, input_dimensions):
    """ convert the 210x160x3 uint8 frame into a 6400 float vector """
    processed_observation = input_observation[35:195] # crop
    processed_observation = downsample(processed_observation)
    processed_observation = remove_color(processed_observation)
    processed_observation = remove_background(processed_observation)
    processed_observation[processed_observation != 0] = 1 # everything else (paddles, ball) just set to 1
    # Convert from 80 x 80 matrix to 1600 x 1 matrix
    processed_observation = processed_observation.astype(np.float).ravel()

    # subtract the previous frame from the current one so we are only processing on changes in the game
    if prev_processed_observation is not None:
        input_observation = processed_observation - prev_processed_observation
    else:
        input_observation = np.zeros(input_dimensions)
    # store the previous frame so we can subtract from it next time
    prev_processed_observations = processed_observation
    return input_observation, prev_processed_observations


def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def relu(vector):
    vector[vector < 0] = 0
    return vector

def apply_neural_nets(observation_matrix, weights):
    """ Based on the observation_matrix and weights, compute the new hidden layer values and the new output layer values"""
    hidden_layer_values = np.dot(weights['1'], observation_matrix)
    hidden_layer_values = relu(hidden_layer_values)
    output_layer_values = np.dot(hidden_layer_values, weights['2'])
    output_layer_values = sigmoid(output_layer_values)
    return hidden_layer_values, output_layer_values

def choose_action(probability):
    random_value = np.random.uniform()
    if random_value < probability:
        # signifies up in openai gym
        return 2
    else:
         # signifies down in openai gym
        return 3

def compute_gradient(gradient_log_p, hidden_layer_values, observation_values, weights):
    """ See here: http://neuralnetworksanddeeplearning.com/chap2.html"""
    delta_L = gradient_log_p
    dC_dw2 = np.dot(hidden_layer_values.T, delta_L).ravel()
    delta_l2 = np.outer(delta_L, weights['2'])
    delta_l2 = relu(delta_l2)
    dC_dw1 = np.dot(delta_l2.T, observation_values)
    return {
        '1': dC_dw1,
        '2': dC_dw2
    }

def update_weights(weights, expectation_g_squared, g_dict, decay_rate, learning_rate):
    """ See here: http://sebastianruder.com/optimizing-gradient-descent/index.html#rmsprop"""
    epsilon = 1e-5
    for layer_name in weights.keys():
        g = g_dict[layer_name]
        expectation_g_squared[layer_name] = decay_rate * expectation_g_squared[layer_name] + (1 - decay_rate) * g**2
        weights[layer_name] += (learning_rate * g)/(np.sqrt(expectation_g_squared[layer_name] + epsilon))
        g_dict[layer_name] = np.zeros_like(weights[layer_name]) # reset batch gradient buffer

def discount_rewards(rewards, gamma):
    """ Actions you took 20 steps before the end result are less important to the overall result than an action you took a step ago.
    This implements that logic by discounting the reward on previous actions based on how long ago they were taken"""
    discounted_rewards = np.zeros_like(rewards)
    running_add = 0
    for t in reversed(xrange(0, rewards.size)):
        if rewards[t] != 0:
            running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + rewards[t]
        discounted_rewards[t] = running_add
    return discounted_rewards

def discount_with_rewards(gradient_log_p, episode_rewards, gamma):
    """ discount the gradient with the normalized rewards """
    discounted_episode_rewards = discount_rewards(episode_rewards, gamma)
    # standardize the rewards to be unit normal (helps control the gradient estimator variance)
    discounted_episode_rewards -= np.mean(discounted_episode_rewards)
    discounted_episode_rewards /= np.std(discounted_episode_rewards)
    return gradient_log_p * discounted_episode_rewards


def main():
    env = gym.make("Pong-v0")
    observation = env.reset() # This gets us the image

    # hyperparameters
    episode_number = 0
    batch_size = 10
    gamma = 0.99 # discount factor for reward
    decay_rate = 0.99
    num_hidden_layer_neurons = 200
    input_dimensions = 80 * 80
    learning_rate = 1e-4

    episode_number = 0
    reward_sum = 0
    running_reward = None
    prev_processed_observations = None

    weights = {
        '1': np.random.randn(num_hidden_layer_neurons, input_dimensions) / np.sqrt(input_dimensions),
        '2': np.random.randn(num_hidden_layer_neurons) / np.sqrt(num_hidden_layer_neurons)
    }

    # To be used with rmsprop algorithm (http://sebastianruder.com/optimizing-gradient-descent/index.html#rmsprop)
    expectation_g_squared = {}
    g_dict = {}
    for layer_name in weights.keys():
        expectation_g_squared[layer_name] = np.zeros_like(weights[layer_name])
        g_dict[layer_name] = np.zeros_like(weights[layer_name])

    episode_hidden_layer_values, episode_observations, episode_gradient_log_ps, episode_rewards = [], [], [], []


    while True:
        env.render()
        processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)
        hidden_layer_values, up_probability = apply_neural_nets(processed_observations, weights)
    
        episode_observations.append(processed_observations)
        episode_hidden_layer_values.append(hidden_layer_values)

        action = choose_action(up_probability)

        # carry out the chosen action
        observation, reward, done, info = env.step(action)

        reward_sum += reward
        episode_rewards.append(reward)

        # see here: http://cs231n.github.io/neural-networks-2/#losses
        fake_label = 1 if action == 2 else 0
        loss_function_gradient = fake_label - up_probability
        episode_gradient_log_ps.append(loss_function_gradient)


        if done: # an episode finished
            episode_number += 1

            # Combine the following values for the episode
            episode_hidden_layer_values = np.vstack(episode_hidden_layer_values)
            episode_observations = np.vstack(episode_observations)
            episode_gradient_log_ps = np.vstack(episode_gradient_log_ps)
            episode_rewards = np.vstack(episode_rewards)

            # Tweak the gradient of the log_ps based on the discounted rewards
            episode_gradient_log_ps_discounted = discount_with_rewards(episode_gradient_log_ps, episode_rewards, gamma)

            gradient = compute_gradient(
              episode_gradient_log_ps_discounted,
              episode_hidden_layer_values,
              episode_observations,
              weights
            )

            # Sum the gradient for use when we hit the batch size
            for layer_name in gradient:
                g_dict[layer_name] += gradient[layer_name]

            if episode_number % batch_size == 0:
                update_weights(weights, expectation_g_squared, g_dict, decay_rate, learning_rate)

            episode_hidden_layer_values, episode_observations, episode_gradient_log_ps, episode_rewards = [], [], [], [] # reset values
            observation = env.reset() # reset env
            running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
            print 'resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward)
            reward_sum = 0
            prev_processed_observations = None

main()